In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install langchain
!pip install -U langchain-community
!pip install sentence-transformers
!pip install faiss-gpu
!pip install pypdf

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter

# Load the document
loader = PyPDFLoader("/content/drive/MyDrive/Asha/book-no-1.pdf")
documents = loader.load()

# Split the document into chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

In [4]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Load embedding model
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
# model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    # model_kwargs=model_kwargs
)

# Create FAISS vector store
vectorstore = FAISS.from_documents(docs, embeddings)

# Save and reload the vector store
vectorstore.save_local("faiss_index_")
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings, allow_dangerous_deserialization=True)

# Create a retriever
retriever = persisted_vectorstore.as_retriever()

<ipython-input-4-4cf1a404cba4>:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [5]:
!pip install langchain_ollama
!pip install colab-xterm
# %xterm
!curl -fsSL ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [9]:
import subprocess

# Start the Ollama serve command in the background
process = subprocess.Popen(["ollama", "serve"], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
print(f"Started Ollama server with PID: {process.pid}")


Started Ollama server with PID: 24336


In [10]:
!ollama pull llama2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 8934d96d3f08... 100% ▕▏ 3.8 GB                         
pulling 8c17c2ebb0ea... 100% ▕▏ 7.0 KB                         
pulling 7c23fb36d801... 100% ▕▏ 4.8 KB                         
pulling 2e0493f67d0c... 100% ▕▏   59 B                         
pulling fa304d675061... 100% ▕▏   91 B                         
pulling 42ba7f8a01dd... 100% ▕▏  557 B                         
verifying sha256 digest 
writing manifest 
success 


In [11]:
from langchain_community.llms import Ollama

# Initialize the LLaMA model
llm = Ollama(model="llama2")

# Test with a sample prompt
response = llm.invoke("Tell me a joke")
print(response)


Why don't scientists trust atoms? Because they make up everything! 😂


In [12]:
from langchain.chains import RetrievalQA

# Create RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

# Interactive query loop
while True:
    query = input("Type your query (or type 'Exit' to quit): \n")
    if query.lower() == "exit":
        break
    result = qa.run(query)
    print(result)

Type your query (or type 'Exit' to quit): 
what are the duties of asha workers?


<ipython-input-12-cb709c31b89b>:11: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa.run(query)


Based on the provided context, the duties of ASHA workers include:

1. Guiding villagers to access good healthcare in a timely and affordable manner.
2. Involving village people in health planning to assess health needs and select appropriate solutions to address common problems.
3. Helping in selecting appropriate solutions to address health issues in the village.
4. Improving the health status of the villagers through teamwork with other stakeholders, including the people, nurse, anganwadi, Dai, schoolteachers, and village Panchayat.
5. Working with doctors to ensure that people remain healthy.
6. Assisting in identifying and addressing the needs of BPL families, which are eligible for special benefits.

Overall, ASHA workers play a crucial role in improving the healthcare situation in rural areas by mobilizing community participation and working collaboratively with other stakeholders to address health issues.


KeyboardInterrupt: Interrupted by user